## Quick Tmpo demo

To get started, clone the tmpo-py repository from the [**opengrid github page**](https://github.com/opengridcc/tmpo-py). Then specify the path to this repo on your hard drive in your `opengrid.cfg` file.

In [ ]:
import sys
import os
import inspect
# add the path to opengrid to sys.path
script_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.append(os.path.join(script_dir, os.pardir, os.pardir))
from opengrid.library import houseprint
from opengrid.library import config
c=config.Config()
sys.path.append(c.get('tmpo', 'folder'))
import tmpo
import pandas as pd

import datetime as dt
import matplotlib.pyplot as plt
import pytz
bxl=pytz.timezone('Europe/Brussels')
%matplotlib inline
plt.rcParams['figure.figsize'] = 14,8

Create a tmpo session, and enter debug mode to get more output.

In [ ]:
s = tmpo.Session()
s.debug = True

Add a sensor and token to start tracking the data for this given sensor.  You only have to do this once for each sensor. 

In [ ]:
s.add('d209e2bbb35b82b83cc0de5e8b84a4ff','e16d9c9543572906a11649d92f902226')

Sync all available data to your hard drive.  All sensors previously added will be synced.

In [ ]:
s.sync()

Now you can create a pandas timeseries with all data from a given sensor.

In [ ]:
ts = s.series('d209e2bbb35b82b83cc0de5e8b84a4ff')
print ts

When plotting the data, you'll notice that this ts contains cumulative data, and the time axis (= pandas index) contains seconds since the epoch.  Not very practical.

In [ ]:
ts.plot()
plt.show()

So the first thing to do is create an DatetimeIndex.

In [ ]:
ts.index = pd.to_datetime((ts.index.values*1e9).astype(int))
ts.plot()
plt.show()

To show differential data (eg instantaneous power), we first have to resample this cumulative data to the interval we want to obtain.  We use linear interpolation to approximate the cumulative value between two datapoints.  In the example below, we resample to minute values. Then, we take the difference between the cumulative values at minute intervals in order to get the average power (per minute). As the original data is in Wh, we have to convert it to W.

In [ ]:
tsmin = ts.resample(rule='60s')
tsmin=tsmin.interpolate(method='linear')
tsmin=tsmin.diff()*3600/60
tsmin.plot()

If we want to plot only a specific period, we can *slice* the data with the *.ix[from:to]* method.

In [ ]:
tsmin.ix['20141016':'20141018'].plot()

In [ ]:
ts.name

## Note

A part of the functionality illustrated above is now included in the fluksoapi module.  See eg. the notebook Demo_WaterGasElekVisualisation.ipynb for a demo.

TMPO support is now included in the new houseprint. After loading the houseprint just call houseprint.init_tmpo() to initialize. After that, accessing data is as easy as calling sensor.get_data(), houseprint.get_data()... there are many options, such as houseprint.get_data(sensortype='gas', resample='min').